In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import datetime

In [2]:
# 한국 전체 종목의 기본 정보 가져오기
korean_market_all_basic = pd.read_csv("./korean_market_data.csv", encoding='euc-kr')
korean_market_all_basic = korean_market_all_basic[['단축코드', '한글 종목명', '상장일', '시장구분']]
korean_market_all_basic = korean_market_all_basic.rename(columns={'단축코드':'종목코드'})
korean_market_all_basic

,종목코드,한글 종목명,상장일,시장구분
0,098120,(주)마이크로컨텍솔루션,2008-09-23,KOSDAQ
1,009520,(주)포스코엠텍,1997-11-10,KOSDAQ
2,095570,AJ네트웍스보통주,2015-08-21,KOSPI
3,006840,AK홀딩스보통주,1999-08-11,KOSPI
4,282330,BGF리테일보통주,2017-12-08,KOSPI
...,...,...,...,...
2702,000547,흥국화재해상보험2우선주(신형),1999-08-09,KOSPI
2703,000540,흥국화재해상보험보통주,1974-12-05,KOSPI
2704,003280,흥아해운보통주,1976-06-29,KOSPI
2705,037440,희림종합건축사사무소,2000-02-03,KOSDAQ


In [3]:
# 코스피정보
kospi = pd.read_csv("./kospi.csv", encoding='euc-kr')
kospi = kospi.drop(['종가', '대비', '등락률', '시장구분'], axis=1)
kospi

,종목코드,종목명,업종명,시가총액
0,095570,AJ네트웍스,서비스업,234579697950
1,006840,AK홀딩스,기타금융,204674817450
2,027410,BGF,기타금융,399617602425
3,282330,BGF리테일,유통업,3099004345800
4,138930,BNK금융지주,기타금융,2024057877660
...,...,...,...,...
938,005010,휴스틸,철강금속,330385881000
939,000540,흥국화재,보험,216497713650
940,000547,흥국화재2우B,보험,3005952000
941,000545,흥국화재우,보험,4992000000


In [4]:
# 코스닥정보
kosdaq = pd.read_csv("./kosdaq.csv", encoding='euc-kr')
kosdaq = kosdaq.drop(['종가', '대비', '등락률', '시장구분'], axis=1)
kosdaq

,종목코드,종목명,업종명,시가총액
0,060310,3S,기계·장비,96345234370
1,054620,APS홀딩스,금융,240243923380
2,265520,AP시스템,반도체,290346999000
3,211270,AP위성,통신장비,253382707200
4,126600,BGF에코머티리얼즈,화학,153586648800
...,...,...,...,...
1625,024060,흥구석유,유통,80850000000
1626,010240,흥국,기계·장비,73936176000
1627,189980,흥국에프엔비,음식료·담배,103354904525
1628,037440,희림,기타서비스,134908782750


In [5]:
# 코스피 코스닥 합치기
kospi_kosdaq = pd.concat([kospi, kosdaq], axis = 0)
kospi_kosdaq

,종목코드,종목명,업종명,시가총액
0,095570,AJ네트웍스,서비스업,234579697950
1,006840,AK홀딩스,기타금융,204674817450
2,027410,BGF,기타금융,399617602425
3,282330,BGF리테일,유통업,3099004345800
4,138930,BNK금융지주,기타금융,2024057877660
...,...,...,...,...
1625,024060,흥구석유,유통,80850000000
1626,010240,흥국,기계·장비,73936176000
1627,189980,흥국에프엔비,음식료·담배,103354904525
1628,037440,희림,기타서비스,134908782750


In [6]:
# 한국 전체
korea_stocks = pd.merge(korean_market_all_basic , kospi_kosdaq, how = "inner", on = "종목코드")
korea_stocks.dtypes

종목코드      object
한글 종목명    object
상장일       object
시장구분      object
종목명       object
업종명       object
시가총액       int64
dtype: object

In [7]:

# 코스피는 뒤에 '.ks', 코스닥은 뒤에 '.kq'
kospi_condition = (korea_stocks['시장구분']=='KOSPI')
kosdaq_condition = (korea_stocks['시장구분']=='KOSDAQ')
korea_stocks.loc[(kospi_condition), "종목코드"] = korea_stocks['종목코드']+".KS"
korea_stocks.loc[(kosdaq_condition), "종목코드"] = korea_stocks['종목코드']+".KQ"
korea_stocks

,종목코드,한글 종목명,상장일,시장구분,종목명,업종명,시가총액
0,098120.KQ,(주)마이크로컨텍솔루션,2008-09-23,KOSDAQ,마이크로컨텍솔,반도체,45803340660
1,009520.KQ,(주)포스코엠텍,1997-11-10,KOSDAQ,포스코엠텍,금속,452656181610
2,095570.KS,AJ네트웍스보통주,2015-08-21,KOSPI,AJ네트웍스,서비스업,234579697950
3,006840.KS,AK홀딩스보통주,1999-08-11,KOSPI,AK홀딩스,기타금융,204674817450
4,282330.KS,BGF리테일보통주,2017-12-08,KOSPI,BGF리테일,유통업,3099004345800
...,...,...,...,...,...,...,...
2568,000547.KS,흥국화재해상보험2우선주(신형),1999-08-09,KOSPI,흥국화재2우B,보험,3005952000
2569,000540.KS,흥국화재해상보험보통주,1974-12-05,KOSPI,흥국화재,보험,216497713650
2570,003280.KS,흥아해운보통주,1976-06-29,KOSPI,흥아해운,운수창고업,324814038549
2571,037440.KQ,희림종합건축사사무소,2000-02-03,KOSDAQ,희림,기타서비스,134908782750


In [8]:
# 나스닥에서 mega랑 large만 필터링에서 다 다운로드받은거(214개 기업)
nasdaq = pd.read_csv("./nasdaq.csv")
nasdaq = nasdaq[['Symbol', 'Name', 'IPO Year', 'Sector', 'Market Cap']]
nasdaq = nasdaq.rename(columns={"Symbol":"종목코드", 'Name':'종목명', 'IPO Year':'상장일', 'Sector':'업종명', 'Market Cap':'시가총액'})
nasdaq['시장구분'] ='NASDAQ'
nasdaq['한글 종목명'] = nasdaq['종목명']
nasdaq

,종목코드,종목명,상장일,업종명,시가총액,시장구분,한글 종목명
0,AAPL,Apple Inc. Common Stock,1980.0,Technology,2.608750e+12,NASDAQ,Apple Inc. Common Stock
1,ABNB,Airbnb Inc. Class A Common Stock,2020.0,Consumer Discretionary,6.915843e+10,NASDAQ,Airbnb Inc. Class A Common Stock
2,ACGL,Arch Capital Group Ltd. Common Stock,NaN,Finance,2.413915e+10,NASDAQ,Arch Capital Group Ltd. Common Stock
3,ADBE,Adobe Inc. Common Stock,1986.0,Technology,1.484508e+11,NASDAQ,Adobe Inc. Common Stock
4,ADI,Analog Devices Inc. Common Stock,NaN,Technology,9.234337e+10,NASDAQ,Analog Devices Inc. Common Stock
...,...,...,...,...,...,...,...
210,XEL,Xcel Energy Inc. Common Stock,NaN,Utilities,3.573533e+10,NASDAQ,Xcel Energy Inc. Common Stock
211,XM,Qualtrics International Inc. Class A Common Stock,2021.0,Technology,1.049940e+10,NASDAQ,Qualtrics International Inc. Class A Common Stock
212,ZBRA,Zebra Technologies Corporation Class A Common ...,1991.0,Technology,1.464624e+10,NASDAQ,Zebra Technologies Corporation Class A Common ...
213,ZM,Zoom Video Communications Inc. Class A Common ...,2019.0,Technology,2.007377e+10,NASDAQ,Zoom Video Communications Inc. Class A Common ...


In [9]:
all_data = pd.concat([korea_stocks, nasdaq], axis=0)
all_data

,종목코드,한글 종목명,상장일,시장구분,종목명,업종명,시가총액
0,098120.KQ,(주)마이크로컨텍솔루션,2008-09-23,KOSDAQ,마이크로컨텍솔,반도체,4.580334e+10
1,009520.KQ,(주)포스코엠텍,1997-11-10,KOSDAQ,포스코엠텍,금속,4.526562e+11
2,095570.KS,AJ네트웍스보통주,2015-08-21,KOSPI,AJ네트웍스,서비스업,2.345797e+11
3,006840.KS,AK홀딩스보통주,1999-08-11,KOSPI,AK홀딩스,기타금융,2.046748e+11
4,282330.KS,BGF리테일보통주,2017-12-08,KOSPI,BGF리테일,유통업,3.099004e+12
...,...,...,...,...,...,...,...
210,XEL,Xcel Energy Inc. Common Stock,NaN,NASDAQ,Xcel Energy Inc. Common Stock,Utilities,3.573533e+10
211,XM,Qualtrics International Inc. Class A Common Stock,2021.0,NASDAQ,Qualtrics International Inc. Class A Common Stock,Technology,1.049940e+10
212,ZBRA,Zebra Technologies Corporation Class A Common ...,1991.0,NASDAQ,Zebra Technologies Corporation Class A Common ...,Technology,1.464624e+10
213,ZM,Zoom Video Communications Inc. Class A Common ...,2019.0,NASDAQ,Zoom Video Communications Inc. Class A Common ...,Technology,2.007377e+10


In [10]:
all_data.to_csv("C:/Users/SSAFY/Desktop/gio/data-preprocessing/all_stocks_basic_data.csv", sep=',', encoding='euc-kr')

OSError: Cannot save file into a non-existent directory: 'C:\Users\SSAFY\Desktop\gio\data-preprocessing'